In [1]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob
from tqdm import tqdm
import os
from multiprocessing import Pool

In [2]:
def update(*a):
    pbar.update()

In [3]:
def repair_mjpeg(f_path):    
    f_size = os.stat(f_path).st_size
    stream = open(f_path, 'rb')
    byte_data = b''
    out = cv2.VideoWriter(f_path + '.fixed.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, (2592, 1944))
    byte_pos = 0
    while byte_pos < f_size - 1025:
        byte_data += stream.read(1024)
        a = byte_data.find(b'\xff\xd8')
        b = byte_data.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = byte_data[a:b+2]
            byte_data = byte_data[b+2:]
            try:
                frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            except:
                continue
            out.write(frame)
        byte_pos = stream.tell()
    out.release()
    stream.close()

In [4]:
gd_path = '/Volumes/GoogleDrive/My Drive/REIP/Data/may_18_RH/*/*/*.avi'
file_list = glob.glob(gd_path, recursive=True)

pool = Pool(4)
pbar = tqdm(total=len(file_list))

for file_path in file_list:
    pool.apply_async(repair_mjpeg, args=(file_path,), callback=update)

pool.close()
pool.join()

  9%|▉         | 11/120 [20:50<2:59:18, 98.70s/it] Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):


In [ ]:
# print('FPS: %f' % cap.get(cv2.CAP_PROP_FPS))
# print('Current loc: %f' % cap.get(cv2.CAP_PROP_POS_MSEC))
# print('%i x %i' % (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
# print('Four CC: %s' % cap.get(cv2.CAP_PROP_FOURCC))
# print('Frame count: %f' % cap.get(cv2.CAP_PROP_FRAME_COUNT))
# print('Convert RGB: %i' % cap.get(cv2.CAP_PROP_CONVERT_RGB))